In [1]:
import numpy as np
import random

from q1_softmax import softmax
from q2_sigmoid import sigmoid, sigmoid_grad
# First implement a gradient checker by filling in the following functions
def gradcheck_naive(f, x):
    """ 
    Gradient check for a function f 
    - f should be a function that takes a single argument and outputs the cost and its gradients
    - x is the point (numpy array) to check the gradient at
    """ 
    
    rndstate = random.getstate()
    random.setstate(rndstate)  
    fx, grad = f(x) # Evaluate function value at original point
    h = 1e-4
    a = []
    # Iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index
        
        ### try modifying x[ix] with h defined above to compute numerical gradients
        ### make sure you call random.setstate(rndstate) before calling f(x) each time, 
        ### this will make it 
        ### possible to test cost functions with built in randomness later
        ### YOUR CODE HERE:
        
        x[ix] += h    
        random.setstate(rndstate)
        f_plus = f(x)[0]    
        x[ix] -= 2 * h    
        random.setstate(rndstate)
        f_minus = f(x)[0]
        numgrad = (f_plus - f_minus) / (2 * h)
        x[ix] += h
        
        ### END YOUR CODE

        # Compare gradients
        reldiff = abs(numgrad - grad[ix]) / max(1, abs(numgrad), abs(grad[ix]))
        
        if reldiff > 1e-5:
            print "Gradient check failed."
            print "First gradient error found at index %s" % str(ix)
            print "Your gradient: %f \t Numerical gradient: %f" % (grad[ix], numgrad)
            return
    
        it.iternext() # Step to next dimension

    print "Gradient check passed!"
    
def sanity_check():
    """
    Some basic sanity checks.
    """
    quad = lambda x: (np.sum(x ** 2), x * 2)
    
    print "Running sanity checks..."
    gradcheck_naive(quad, np.array(123.456))      # scalar test
    gradcheck_naive(quad, np.random.randn(3,))    # 1-D test
    gradcheck_naive(quad, np.random.randn(4,5))   # 2-D test
    print ""
    
def forward_backward_prop(data, labels, params, dimensions):
    """ 
    Forward and backward propagation for a two-layer sigmoidal network 
    
    Compute the forward propagation and for the cross entropy cost,
    and backward propagation for the gradients for all parameters.
    """

    ### Unpack network parameters (do not modify)
    ofs = 0
    Dx, H, Dy = (dimensions[0], dimensions[1], dimensions[2])

    W1 = np.reshape(params[ofs:ofs+ Dx * H], (Dx, H))
    
    ofs += Dx * H
    b1 = np.reshape(params[ofs:ofs + H], (1, H))
    ofs += H
    W2 = np.reshape(params[ofs:ofs + H * Dy], (H, Dy))
    ofs += H * Dy
    b2 = np.reshape(params[ofs:ofs + Dy], (1, Dy))

    ### YOUR CODE HERE: forward propagation
    # forward
    input_layer = data
    N = data.shape[0]
    
    hidden_layer = np.dot(data,W1) + np.tile(b1,(N,1))
    hidden_layer_value = sigmoid(hidden_layer) 
    
    output_layer = np.dot(hidden_layer_value,W2) + np.tile(b2,(N,1))
    output_layer_value = softmax(output_layer)
    
    ## cost function
    cost2 = - np.sum(labels * np.log(output_layer_value)) 

    ### END YOUR CODE
    
    ### YOUR CODE HERE: backward propagation
    
    output_delta = (output_layer_value - labels) 
    
    gradW2 = np.dot(hidden_layer_value.T,output_delta)   
    gradb2 = np.sum(output_delta,axis=0)

    input_delta = np.dot(output_delta,W2.T) * sigmoid_grad(hidden_layer_value)
    
    gradW1 = np.dot(input_layer.T,input_delta)
    gradb1 = np.sum(input_delta,axis=0)
    
    ### END YOUR CODE
    
    ### Stack gradients (do not modify)
    grad = np.concatenate((gradW1.flatten(), gradb1.flatten(), 
        gradW2.flatten(), gradb2.flatten()))

    return cost2, grad

def your_sanity_checks(): 
    """
    Use this space add any additional sanity checks by running:
        python q2_gradcheck.py 
    This function will not be called by the autograder, nor will
    your additional tests be graded.
    """
    print "Running your sanity checks..."
    ### YOUR CODE HERE
    N = 20
    dimensions = [10, 5, 10]
    data = np.random.randn(N, dimensions[0])   # each row will be a datum
    labels = np.zeros((N, dimensions[2]))
    for i in xrange(N):
        labels[i,random.randint(0,dimensions[2]-1)] = 1
    
    params = np.random.randn((dimensions[0] + 1) * dimensions[1] + (
        dimensions[1] + 1) * dimensions[2], )
#     print params.shape
    quad = lambda params: forward_backward_prop(data, labels, params, dimensions)
    
    gradcheck_naive(quad, params)
    ### END YOUR CODE

if __name__ == "__main__":
#     sanity_check()
    your_sanity_checks()


Running your sanity checks...
Gradient check passed!
